In [5]:
import pandas as pd
years_to_include = ("2012","2013","2014","2015","2016","2017","2018")

In [6]:
def unemployment_rate(year):
    """
    Function returns average unemployment rate for each county for each two-digit year passed as a string
    """
    
    # import US BLS labor stats; ignore noise rows at file start and improve column names
    df = pd.read_excel('./data/labor_force_xls/laucnty' + year + '.xlsx', header=4, usecols=["County Name/State Abbreviation","(%)"])
    df.columns = ("County","%Unemployed")

    # find only CT counties
    ct_mask = df["County"].str[-2:] == "CT"
    df = df[ct_mask]

    # strip to county name and set index
    df["County"] = df["County"].str[:-11]
    return df.set_index("County")
    

In [11]:
def create_composite(years):
    
    """
    Function returns a composite of population, economic, and unemployment data for each county by year for desired list of years
    """
    
    
    # load population data for all counties
    census_CT = pd.read_csv('data/census_CT.csv')
    census_CT.loc[0,"County"] = "Connecticut"
    census_CT.set_index("County", inplace=True)
    
    # load death data    
    death_data = pd.read_csv('data/deaths_by_county.csv').groupby("Year")
    
    # create new DataFrame to fill
    composite = pd.DataFrame()
    
    for year in years:
            
        # load economic indicators for given year
        econ = pd.read_csv('data/income_poverty_csv/est' + year[-2:] + '-ct.csv')
        econ.set_index('Jurisdiction', inplace=True)

        # get unemployment rates for given year
        unemp = unemployment_rate(year[-2:])

        # join unemployment and population data on jurisdiction/county name
        econ = econ.join(unemp)
        econ = econ.join(census_CT[year])
        
        # relabel population column
        econ.rename(columns = {
            year : "Population"
        }, inplace=True)
                
        """
        Section calculates statewide unemployment from weighted county unemp and pop
        """        
        df = econ.query('Jurisdiction != "Connecticut"')
        percentages = list(df["%Unemployed"])
        pops = list(df["Population"])
        
        unemp_sum = 0
        i = 0
        while i < len(pops):
            unemp_sum += pops[i] * percentages[i]
            i += 1
        
        econ.loc["Connecticut","%Unemployed"] = unemp_sum / econ.loc["Connecticut"]["Population"]
        """
        End subroutine
        """
        
        # add state-normalized poverty and unemployment levels
        econ["povNorm"] = (econ['%inPoverty'] / econ.loc["Connecticut","%inPoverty"] * 100).astype('int')
        econ["povMinorsNorm"] = (econ['%MinorsInPoverty'] / econ.loc["Connecticut","%MinorsInPoverty"] * 100).astype('int')
        econ["unempNorm"] = (econ['%Unemployed'] / econ.loc["Connecticut","%Unemployed"] * 100).astype('int')
        econ["incomeNorm"] = (econ["MedianHouseholdIncome"] / econ.loc["Connecticut","MedianHouseholdIncome"] * 100).astype('int')
        
        # join death data by county, sum for state
        year_death_counties = death_data.get_group(int(year)).set_index("Jurisdiction")
        econ = econ.join(year_death_counties[["DeathsInCounty","ResidentDeaths"]])
        econ.loc["Connecticut","ResidentDeaths"] = year_death_counties["ResidentDeaths"].sum()
        econ.loc["Connecticut","DeathsInCounty"] = year_death_counties["DeathsInCounty"].sum()
        
        # calculate per cap death rates & normalized rates
        econ["PerCapDeathsInCounty"] = econ["DeathsInCounty"] / econ["Population"]
        econ["PerCapResidentDeaths"] = econ["ResidentDeaths"] / econ["Population"]
        econ["DeathsInCountyNorm"] = (econ["PerCapDeathsInCounty"] / econ.loc["Connecticut"]["PerCapDeathsInCounty"] * 100)
        econ["ResidentDeathsNorm"] = (econ["PerCapResidentDeaths"] / econ.loc["Connecticut"]["PerCapResidentDeaths"] * 100)
                
        # add new column "Year" with current year; reorder columns
        econ["Year"] = year
        econ = econ[list(econ.columns)[-1:]+list(econ.columns)[0:-1]]
        
        # concatenate current year to composite
        composite = pd.concat([composite, econ])
        
    # return composite with numbered index
    return composite.reset_index()
                

In [12]:
# create composite and write to CSV
CT_econ_2012_through_2018 = create_composite(years_to_include)
CT_econ_2012_through_2018.to_csv('data/CT_composite.csv')
CT_econ_2012_through_2018

,Jurisdiction,Year,MedianHouseholdIncome,%inPoverty,TotalInPoverty,%MinorsInPoverty,TotalMinorsInPoverty,%Unemployed,Population,povNorm,povMinorsNorm,unempNorm,incomeNorm,DeathsInCounty,ResidentDeaths,PerCapDeathsInCounty,PerCapResidentDeaths,DeathsInCountyNorm,ResidentDeathsNorm
0,Connecticut,2012,67275,10.6,370537,14.6,114613,8.349387,3594547,100,100,100,100,355.0,347.0,0.000099,0.000097,100.000000,100.000000
1,Fairfield,2012,79536,8.9,81756,11.1,24854,7.800000,935099,83,76,93,118,52.0,51.0,0.000056,0.000055,56.306893,56.497246
2,Hartford,2012,63374,12.2,106357,16.7,32889,8.500000,897706,115,114,101,94,101.0,92.0,0.000113,0.000102,113.920809,106.161829
3,Litchfield,2012,67746,7.0,12975,8.6,3272,7.600000,187570,66,58,91,100,17.0,20.0,0.000091,0.000107,91.770132,110.453965
4,Middlesex,2012,74588,6.4,10376,7.5,2470,7.200000,165634,60,51,86,110,19.0,26.0,0.000115,0.000157,116.150190,162.606743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Middlesex,2018,88709,6.7,10556,8.1,2314,3.500000,162870,65,58,84,116,44.0,87.0,0.000270,0.000534,94.780040,193.881399
59,New Haven,2018,67720,11.6,96563,16.7,28530,4.400000,856971,112,121,106,88,269.0,239.0,0.000314,0.000279,110.126405,101.225461
60,New London,2018,67790,9.8,25063,12.5,6388,4.000000,266285,95,90,96,88,91.0,89.0,0.000342,0.000334,119.894703,121.311310
61,Tolland,2018,89584,8.0,10835,6.4,1652,3.600000,150689,77,46,86,117,26.0,33.0,0.000173,0.000219,60.533684,79.485951
